In [269]:
import pandas as pd
import numpy as np
import csv

nfb = pd.read_csv('NeuroFB_part5.csv', low_memory=False)
nfb.head()

val = nfb.iloc[0].values
nfb=nfb.loc[1:,]

nfb.columns

ident_dict = dict(zip(nfb.columns.get_values(), val.T))

remove_keys = [] 
for key, value in ident_dict.items(): 
    if(type(value) == float or type(value) == np.float_):
        if(np.isnan(value)): 
            remove_keys.append(key) 
print(remove_keys)

for i in range(len(remove_keys)):
    if remove_keys[i] in ident_dict:
        del ident_dict[remove_keys[i]]

nfb.set_index('queried_ursi', inplace=True)

['queried_ursi', 'oldest_ursi', 'instrument_id', 'instrument_label', 'visit', 'visit_instance', 'asmt_ursi', 'assessment_date', 'assessment_id', 'entry_code', 'site_id', 'source_type', 'rater1', 'successfully_completed', 'notes', 'instrument_id.1', 'instrument_label.1', 'visit.1', 'visit_instance.1', 'asmt_ursi.1', 'assessment_date.1', 'assessment_id.1', 'entry_code.1', 'site_id.1', 'source_type.1', 'rater1.1', 'successfully_completed.1', 'notes.1', 'instrument_id.2', 'instrument_label.2', 'visit.2', 'visit_instance.2', 'asmt_ursi.2', 'assessment_date.2', 'assessment_id.2', 'entry_code.2', 'site_id.2', 'source_type.2', 'rater1.2', 'successfully_completed.2', 'notes.2', 'instrument_id.3', 'instrument_label.3', 'visit.3', 'visit_instance.3', 'asmt_ursi.3', 'assessment_date.3', 'assessment_id.3', 'entry_code.3', 'site_id.3', 'source_type.3', 'rater1.3', 'successfully_completed.3', 'notes.3', 'instrument_id.4', 'instrument_label.4', 'visit.4', 'visit_instance.4', 'asmt_ursi.4', 'assessment

In [270]:
#Codebook reading
pre_cb = {}
codebook_lines = []
with open('CB.csv') as csvfile:
    csvreader = csv.reader(csvfile, dialect='excel')
    for row in csvreader:
        codebook_lines.append(row)

current_key = ''
for line_vals in codebook_lines:
    if 'Instrument' in line_vals[0]:
        continue
    
    if line_vals[3]:
        if line_vals[3] in pre_cb:
            raise ValueError('Key {0} already exists in dictionary!'.format(line_vals[3]))
        pre_cb[line_vals[3]] = {}
        current_key = line_vals[3]

    if not current_key:
        raise ValueError("Trying to set value with empty key")
    pre_cb[current_key][line_vals[5]] = line_vals[4]

In [271]:
#Instrument keys readin from codebook
instrument_dict = {}
current_instrmt = ''
previous_instrmt = ''
for line_vals in codebook_lines:
    if 'Instrument' in line_vals[0]:
        continue
    previous_instrmt = ''    
    if line_vals[0]:
        if line_vals[0] in instrument_dict:
            raise ValueError('Error')
        current_instrmt = line_vals[0]
        instrument_dict.setdefault(current_instrmt, [])
    while(current_instrmt!=previous_instrmt):
        if line_vals[3]:
            instrument_dict[current_instrmt].append(line_vals[3])
        previous_instrmt = current_instrmt

In [272]:
def drop_NanRows(df):
    drop_row = True
    drop_list = []
    for i in range(len(df.index)):
        row = df.iloc[i]
        drop_row = True
        for f in range(len(row)-2):
            if(type(row[f+2]) == float or type(row[f+2]) == np.float64):
                if not(np.isnan(row[f+2])):
                    drop_row = False
            elif(type(row[f+2]) == str and row[f+2].lower() != 'nan'):
                drop_row = False
            else:
                drop_row = False
        if(drop_row):
            drop_list.append(i)
    return drop_list

In [273]:
def convertToCodebook(k, value):
    if(type(value)==str and value.lower()!='nan'):
        if k in pre_cb.keys():
            if value in pre_cb[k]:
                return pre_cb[k][value]
            elif value not in pre_cb[k]:
                return value 
        else:
            return value
    elif((type(value) == float or type(value) == np.float64) and np.isnan(value)):
        return value

In [274]:
def get_instrumentID(column_name, var_id):
    inst_id = ''
    found = False
    if not var_id:
        for inst, variables in instrument_dict.items():
            for val in variables:
                if(column_name == val):
                    inst_id = inst
                    found = True
    if (not found) or var_id:
        if((column_name[column_name.rfind('_', 0, column_name.rfind('_'))+1:][0]).isdigit()): #if first character after the last '_' is a digit
            inst_id = column_name[:(column_name.rfind('_', 0, column_name.rfind('_')))]
        else:
            inst_id = column_name[:(column_name.rfind('_'))]
        for inst, variables in instrument_dict.items():             
            if (inst_id == get_instrumentID(variables[0], not var_id)):
                inst_id = inst
    return inst_id

In [275]:
#List of possible visit_ids
visitID_list = ['V1', 'V1REP', 'V1REP_2', 'V2', 'VA', 'VA_ALG', 
                'V2REP', 'V2REP_2', 'VA-REP', 'V3', 'V4', 'V5']

In [276]:
#Try to make unique list using visitID_list
new_col = ['queried_ursi', 'visit_id']
processed = []
nfb_col;
for c in nfb.columns:
    for visit in (sorted(visitID_list, key=len, reverse=True)):
        if(visit+'_' in c) and (c not in processed):
            nfb_col = c.replace(visit+'_', '')
            if(nfb_col[0].isdigit()):
                nfb_col = nfb_col[nfb_col.find('_')+1:]
            new_col.append(nfb_col)
            processed.append(c)
ist = [x for i, x in enumerate(new_col) if new_col.index(x) == i]

In [277]:
#Try to make unique list using visitID_list
def get_VisitID(column_name):
    ncol = ''
    visit = ''
    for v in (sorted(visitID_list, key=len, reverse=True)):
        if(v+'_' in column_name):
            ncol = column_name.replace(v+'_', '')
            visit = v
            if(ncol[0].isdigit()):
                ncol = ncol[ncol.find('_')+1:]
                visit = column_name[:column_name.find(ncol)-1]
        
    return visit, ncol

In [67]:
###SKIP###
'''new_col = ['queried_ursi', 'visit_id']
nfb_col;
for key in pre_cb:
    for c in nfb.columns:
        if ('_'+key+'_') in c:   
            nfb_col = c[(c.find('_'+key)+1):]
            if ('_NFB3_MRIQ_') in c: #For Part5
                nfb_col = c[(c.find('_')+1):]
                print(nfb_col)
        elif (c[0]=='V'):
            if ((c[c.find('_')+1:])[0].isdigit()): #if first character after the first '_' is a digit
                nfb_col = c[c.find('_', c.find('_') + 1)+1:]
            elif (c[c.rfind('_', 0, c.rfind('_'))+1:])[0].isdigit():
                nfb_col = c[c.rfind('_', 0, c.rfind('_', 0, c.rfind(c[c.rfind('_', 0, c.rfind('_')):]))+1)+1:]
            else:
                nfb_col = c[c.rfind('_', 0, c.rfind('_'))+1:]
        new_col.append(nfb_col)
unique_list = [x for i, x in enumerate(new_col) if new_col.index(x) == i]'''

In [278]:
new_df = pd.DataFrame(columns=ist)
columns = nfb.columns.sort_values()

Index(['V1REP_2_NEFFI_23', 'V1REP_2_NEOFFI_01', 'V1REP_2_NEOFFI_02',
       'V1REP_2_NEOFFI_03', 'V1REP_2_NEOFFI_04', 'V1REP_2_NEOFFI_05',
       'V1REP_2_NEOFFI_06', 'V1REP_2_NEOFFI_07', 'V1REP_2_NEOFFI_08',
       'V1REP_2_NEOFFI_09',
       ...
       'visit_instance.43', 'visit_instance.44', 'visit_instance.45',
       'visit_instance.46', 'visit_instance.47', 'visit_instance.5',
       'visit_instance.6', 'visit_instance.7', 'visit_instance.8',
       'visit_instance.9'],
      dtype='object', length=4205)

In [280]:
def create_instrumentDF(instrmt):
    #get column names
    a_columns = ['queried_ursi','visit_id']
    for c in nfb_part.columns:
        if instrmt in c:
            a_columns.append(c)
    df = pd.DataFrame(columns=a_columns)
    
    #add to dataframe
    for i in range(len(nfb_part.index)):
        for c in a_columns:
            if instrmt in c:
                df.at[i, 'queried_ursi'] = nfb_part.queried_ursi[i]
                df.at[i, 'visit_id'] = nfb_part.visit_id[i]
                df.at[i, c] = nfb_part.loc[i, c]
    #Drop nan rows
    df = df.drop(drop_NanRows(df))
    df.set_index('queried_ursi', inplace=True)
    return df

In [281]:
#Dataframe restructuring WITH codebook converting
row_counter = -1
target_index_counter = -1
previous_visit = ''

for i in range(len(nfb.index)):
    previous_visit = ''
    row_counter+=1
    for c in columns:
        for key in ist:
            if key in c:
                visit_col, nfb_3_col = get_VisitID(c)
                if(visit_col == 'V3_NFB3'):
                    print('{0} {1} {2}'.format(visit_col, c, key))
                if (previous_visit != visit_col):
                    target_index_counter+=1
                    new_df.at[target_index_counter, 'visit_id'] = visit_col
                    new_df.at[target_index_counter, 'queried_ursi'] = nfb.index[row_counter]
                    previous_visit = visit_col
                val = nfb.loc[nfb.index[row_counter],c]
                new_df.at[target_index_counter, nfb_3_col] = convertToCodebook(key, val)


In [288]:
#Drop nan rows of dataframe
drop_NanRows(new_df)
new_df = new_df.drop(drop_NanRows(new_df))
nfb_part = new_df.reset_index(drop=True)
nfb_part = nfb_part.dropna(axis=1, how='all')

In [310]:
#create dictionary according to instrument/assessment
previous_instrument = ''
instrument = ''
instrumentDF_dict = {}
instrument_id = ''

for c in nfb_part.columns: 
    if(c != 'queried_ursi' and c != 'visit_id'):
        instrument = c[:(c.find('_'))] 
        instrument_id = get_instrumentID(c, False)
    if (previous_instrument != instrument):
        instrumentDF_dict[instrument_id] = create_instrumentDF(instrument)
        previous_instrument = instrument

In [312]:
instrumentDF_dict.keys()

dict_keys(['NEO-FFI-3', 'NKI', 'Penn CNP (12/18/13)', 'NIDA (3/19/13)', 'PDI-21', 'Pegboard', 'PhenX Sexual History', 'PSQI', 'List Learning', 'Debfrief', 'Debrief', 'NFB3_MRIQ', 'PANAS-S', 'CCDPSWQ', 'PTQ', 'QME', 'RRS', 'RSQ', 'RVIP', 'PASE'])